5/14 (Tue) | Experiment

# Preliminary Analyses of Annoation

## 1. Introduction

This notebook conducts preliminary analyses.
The goal of current analyses is to fill the following table.

| Task | WER | N disfluency (Manual / Automatic) | N MCP (Manual / Automatic) | N ECP (Manual / Automatic) |
| - | - | - | - | - |
| Arg_Oly |  |  |  |  |
| Cartoon |  |  |  |  |
| RtSwithoutRAA |  |  |  |  |
| RtSwithRAA |  |  |  |  |
| Monologue |  |  |  |  |
| WoZ_Interview |  |  |  |  |
| ALL |  |  |  |  |

Before starting the analyses, the following code block loads required packages and define global variables.

In [1]:
from typing import List, Tuple, Dict, Generator, Optional
from pathlib import Path

import numpy as np
import pandas as pd
from jiwer import wer

from utils.mfr import logit_2_rating

DATA_DIR = Path("/home/matsuura/Development/app/feature_extraction_api/experiment/data")

MONOLOGUE_TASK = ["Arg_Oly", "Cartoon", "RtSwithoutRAA", "RtSwithRAA"]
DIALOGUE_TASK = ["WoZ_Interview"]

FILLER = {"uh", "ah", "um", "mm", "hmm", "oh", "mm-hmm", "er", "mhm", "uh-huh", "er", "erm", "huh", "uhu", "mmhmm", "uhhuh"}

---

## 2. Define Functions

This section defines functions for the preliminary analyses.
The following code block defines two functions; one generates csv file paths of manual and automatic annotation results; and another one loads them.

In [2]:
def annotation_result_csv_path_generator(task: str, rating_filter: Optional[List[int]] =None) -> Generator[Tuple[Path, Path], None, None]:
    load_dir = DATA_DIR / f"{task}/10_SCTK_Inputs"

    if rating_filter is None:
        for manu_csv_path in load_dir.glob("*_manu.csv"):
            filename = manu_csv_path.stem.removesuffix("_manu")
            auto_csv_path = load_dir / f"{filename}_auto.csv"

            yield manu_csv_path, auto_csv_path
    else:
        pf_path = DATA_DIR / f"{task}/12_PF_Rating/pf_rating.csv"
        df_pf = pd.read_csv(pf_path)
        uid_list = df_pf["uid"].to_numpy()

        logit_path = pf_path.parent / "logit_all.csv"
        threshold_path = logit_path.parent / "threshold_all.csv"
        if task == "WoZ_Interview":
            logit_path = pf_path.parent / "logit.csv"
            threshold_path = logit_path.parent / "threshold.csv"
        
        df_logit = pd.read_csv(logit_path, index_col=0)
        rating_list = logit_2_rating(df_logit["theta"], threshold_path)

        mask = np.full(rating_list.shape, False, dtype=bool)
        for rating in rating_filter:
            mask = mask | (rating_list == rating)
        
        uid_list = uid_list[mask]

        for uid in uid_list:
            if task == "WoZ_Interview":
                uid = str(int(uid)).zfill(3)

            if task == "RtSwithoutRAA":
                if uid == "2055_RtSwithoutRAA":
                    continue

            filename_pattern = f"{uid}*_manu.csv"
            for manu_csv_path in load_dir.glob(filename_pattern):
                filename = manu_csv_path.stem.removesuffix("_manu")
                auto_csv_path = load_dir / f"{filename}_auto.csv"

                yield manu_csv_path, auto_csv_path

def load_dataset(
        rating_filter_monologue: Optional[List[int]] =None,
        rating_filter_dialogue: Optional[List[int]] =None,
) -> Dict[str, Dict[str, List[Dict[str, pd.DataFrame]]]]:
    dataset = {
        "monologue": {},
        "dialogue": {}
    }
    
    for monologue_task in MONOLOGUE_TASK:
        dataset["monologue"][monologue_task] = []
        
        for manu_csv_path, auto_csv_path in annotation_result_csv_path_generator(monologue_task, rating_filter=rating_filter_monologue):
            df_manu = pd.read_csv(manu_csv_path)
            df_auto = pd.DataFrame([], columns=["text"])
            if auto_csv_path.exists():
                df_auto = pd.read_csv(auto_csv_path)

            dataset["monologue"][monologue_task].append({
                "manual": df_manu,
                "automatic": df_auto
            })

    for dialogue_task in DIALOGUE_TASK:
        dataset["dialogue"][dialogue_task] = []

        for manu_csv_path, auto_csv_path in annotation_result_csv_path_generator(dialogue_task, rating_filter=rating_filter_dialogue):
            df_manu = pd.read_csv(manu_csv_path, na_values=["", " "], keep_default_na=False)
            df_auto = pd.DataFrame([], columns=["text"])
            if auto_csv_path.exists():
                df_auto = pd.read_csv(auto_csv_path, na_values=["", " "], keep_default_na=False)

            dataset["dialogue"][dialogue_task].append({
                "manual": df_manu,
                "automatic": df_auto
            })

    return dataset

The following code block defines a function to calculate WER.

In [3]:
def calculate_wer(annotation_results: List[Dict[str, pd.DataFrame]], remove_filer: bool =False) -> float:
    ref = []
    hyp = []

    for annotation_result in annotation_results:
        df_manu = annotation_result["manual"]
        df_auto = annotation_result["automatic"]

        mask_tag_manu = df_manu["text"].astype(str).str.endswith(">")
        mask_tag_auto = df_auto["text"].astype(str).str.endswith(">")

        df_manu = df_manu[~mask_tag_manu]
        df_auto = df_auto[~mask_tag_auto]

        if remove_filer:
            for filler in FILLER:
                mask_filler_manu = (df_manu["text"] == filler)
                df_manu = df_manu[~mask_filler_manu]

                mask_filler_auto = (df_auto["text"] == filler)
                df_auto = df_auto[~mask_filler_auto]

        text_manu = " ".join(df_manu["text"].astype(str))
        text_auto = " ".join(df_auto["text"].astype(str))

        if len(text_manu) == 0 or len(text_auto) == 0:
            continue

        ref.append(text_manu)
        hyp.append(text_auto)

    return wer(ref, hyp)

The following code block defines a function to count the number of tags.

In [4]:
def count_tags(annotation_results: List[Dict[str, pd.DataFrame]], target_tag: str) -> Tuple[List[int], List[int]]:
    n_tag_manu = []
    n_tag_auto = []

    for annotation_result in annotation_results:
        df_manu = annotation_result["manual"]
        df_auto = annotation_result["automatic"]

        mask_tag_manu = (df_manu["text"] == target_tag)
        mask_tag_auto = (df_auto["text"] == target_tag)

        n_tag_manu.append(mask_tag_manu.sum())
        n_tag_auto.append(mask_tag_auto.sum())

    return n_tag_manu, n_tag_auto

---

## 3. Preliminary Analyses

This section conducts the preliminary analyses.
The following code block loads entire dataset.

In [5]:
dataset = load_dataset()

In [6]:
dataset["monologue"]["Arg_Oly"][0]["manual"].head()

,start_time,end_time,type,text
0,0.220000,0.240063,01_text,i
1,0.300063,0.660125,01_text,agree
2,0.820187,1.040250,01_text,this
3,1.260312,2.080563,01_text,statement
4,2.132178,2.764380,02_pause,<CE>


In [7]:
dataset["monologue"]["Arg_Oly"][0]["automatic"].head()

,start_time,end_time,type,text
0,0.220000,0.240063,01_text,i
1,0.300063,0.660125,01_text,agree
2,0.820187,1.040250,01_text,this
3,1.260312,2.080563,01_text,statement
4,2.080563,2.740750,02_pause,<CI>


### 3.1. WER

The following code block calculate WER of monologue tasks.

In [8]:
monologue_data = []
for monologue_task in MONOLOGUE_TASK:
    annotation_results = dataset["monologue"][monologue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {monologue_task} = {res}")

    monologue_data += annotation_results

res = calculate_wer(monologue_data, remove_filer=True)
print(f"WER of monologue task = {res}")

WER of Arg_Oly = 0.1602270395951583
WER of Cartoon = 0.13862017646433508
WER of RtSwithoutRAA = 0.1987206157686707
WER of RtSwithRAA = 0.20201547971533945
WER of monologue task = 0.17605261694198787


The following code block calculate WER of a dialogue task.

In [9]:
dialogue_data = []
for dialogue_task in DIALOGUE_TASK:
    annotation_results = dataset["dialogue"][dialogue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {dialogue_task} = {res}")

    dialogue_data += annotation_results

WER of WoZ_Interview = 0.15021711724331138


The following code block calcualte WER of the entire tasks.

In [10]:
all_task_data = monologue_data + dialogue_data

res = calculate_wer(all_task_data, remove_filer=True)
print(f"WER of all tasks = {res}")

WER of all tasks = 0.1674828781444276


### 3.2. Count Disfluency

The following code block counts the number of disfluency words in monologue tasks

In [11]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<DISFLUENCY>")
print(f"[Manual] N_disfluency of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_disfluency of monologue task = {sum(n_tags_auto)}")

[Manual] N_disfluency of Arg_Oly = 1876
[Automatic] N_disfluency of Arg_Oly = 1793
[Manual] N_disfluency of Cartoon = 2813
[Automatic] N_disfluency of Cartoon = 2517
[Manual] N_disfluency of RtSwithoutRAA = 2949
[Automatic] N_disfluency of RtSwithoutRAA = 2542
[Manual] N_disfluency of RtSwithRAA = 2887
[Automatic] N_disfluency of RtSwithRAA = 2556
[Manual] N_disfluency of monologue task = 10525
[Automatic] N_disfluency of monologue task = 9408


The following code block counts the number of disfluency words in dialogue tasks.

In [12]:
for dialogue_task in DIALOGUE_TASK:
    annotation_results = dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_disfluency of WoZ_Interview = 3935
[Automatic] N_disfluency of WoZ_Interview = 3514


The following code block counts the number of disfluency words in the whole tasks.

In [13]:
n_tags_manu, n_tags_auto = count_tags(all_task_data, "<DISFLUENCY>")
print(f"[Manual] N_disfluency of all tasks = {sum(n_tags_manu)}")
print(f"[Automatic] N_disfluency of all tasks = {sum(n_tags_auto)}")

[Manual] N_disfluency of all tasks = 14460
[Automatic] N_disfluency of all tasks = 12922


### 3.3. Count Mid-Clause Pauses

The following code block counts the number of MCP in monologue tasks

In [14]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CI>")
print(f"[Manual] N_MCP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_MCP of monologue task = {sum(n_tags_auto)}")

[Manual] N_MCP of Arg_Oly = 4302
[Automatic] N_MCP of Arg_Oly = 4923
[Manual] N_MCP of Cartoon = 5559
[Automatic] N_MCP of Cartoon = 6739
[Manual] N_MCP of RtSwithoutRAA = 6169
[Automatic] N_MCP of RtSwithoutRAA = 7367
[Manual] N_MCP of RtSwithRAA = 6332
[Automatic] N_MCP of RtSwithRAA = 7502
[Manual] N_MCP of monologue task = 22362
[Automatic] N_MCP of monologue task = 26531


The following code block counts the number of MCP in dialogue tasks.

In [15]:
for dialogue_task in DIALOGUE_TASK:
    annotation_results = dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_MCP of WoZ_Interview = 7574
[Automatic] N_MCP of WoZ_Interview = 10322


The following code block counts the number of disfluency words in the whole tasks.

In [16]:
n_tags_manu, n_tags_auto = count_tags(all_task_data, "<CI>")
print(f"[Manual] N_MCP of all tasks = {sum(n_tags_manu)}")
print(f"[Automatic] N_MCP of all tasks = {sum(n_tags_auto)}")

[Manual] N_MCP of all tasks = 29936
[Automatic] N_MCP of all tasks = 36853


### 3.4. Count End-Clause Pauses

The following code block counts the number of ECP in monologue tasks

In [17]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CE>")
print(f"[Manual] N_ECP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_ECP of monologue task = {sum(n_tags_auto)}")

[Manual] N_ECP of Arg_Oly = 1142
[Automatic] N_ECP of Arg_Oly = 1001
[Manual] N_ECP of Cartoon = 1985
[Automatic] N_ECP of Cartoon = 1810
[Manual] N_ECP of RtSwithoutRAA = 1835
[Automatic] N_ECP of RtSwithoutRAA = 1627
[Manual] N_ECP of RtSwithRAA = 1842
[Automatic] N_ECP of RtSwithRAA = 1573


[Manual] N_ECP of monologue task = 6804
[Automatic] N_ECP of monologue task = 6011


The following code block counts the number of ECP in dialogue tasks.

In [18]:
for dialogue_task in DIALOGUE_TASK:
    annotation_results = dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_ECP of WoZ_Interview = 2414
[Automatic] N_ECP of WoZ_Interview = 2646


The following code block counts the number of disfluency words in the whole tasks.

In [19]:
n_tags_manu, n_tags_auto = count_tags(all_task_data, "<CE>")
print(f"[Manual] N_ECP of all tasks = {sum(n_tags_manu)}")
print(f"[Automatic] N_ECP of all tasks = {sum(n_tags_auto)}")

[Manual] N_ECP of all tasks = 9218
[Automatic] N_ECP of all tasks = 8657


---

## 4. Summary

As results, the following table was obtained.

| Task | WER | N disfluency (Manual / Automatic) | N MCP (Manual / Automatic) | N ECP (Manual / Automatic) |
| - | - | - | - | - |
| Arg_Oly | 16.1% | 1,876 / 1,793 | 4,302 / 4,923 | 1,142 / 1,001 |
| Cartoon | 13.9% | 2,813 / 2,517 | 5,559 / 6,739 | 1,985 / 1,810 |
| RtSwithoutRAA | 19.9% | 2,949 / 2,542 | 6,169 / 7,367 | 1,835 / 1,627 |
| RtSwithRAA | 20.2% | 2,887 / 2,556 | 6,332 / 7,502 | 1,842 / 1,573 |
| Monologue | 17.6% | 10,525 / 9,408 | 22,362 / 26,531 | 6,804 / 6,011 |
| WoZ_Interview | 15.0% | 3,935 / 3,514 | 7,574 / 10,322 | 2,414 / 2,646 |
| ALL | 16.8% | 14,460 / 12,922 | 29,936 / 36,853 | 9,218 / 8,657 |

---

## 5. Additional Analyses

This section conducts the same analyses for each PF groups.

### 5.1. Beginners

The following code block loads beginners' speech.

In [20]:
beginner_dataset = load_dataset(rating_filter_monologue=[0, 1, 2], rating_filter_dialogue=[0, 1])

The following code block calculates WER of beginners' speech.

In [21]:
monologue_data = []
for monologue_task in MONOLOGUE_TASK:
    annotation_results = beginner_dataset["monologue"][monologue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {monologue_task} = {res}")

    monologue_data += annotation_results

res = calculate_wer(monologue_data, remove_filer=True)
print(f"WER of monologue task = {res}")

dialogue_data = []
for dialogue_task in DIALOGUE_TASK:
    annotation_results = beginner_dataset["dialogue"][dialogue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {dialogue_task} = {res}")

    dialogue_data += annotation_results

all_task_data = monologue_data + dialogue_data

res = calculate_wer(all_task_data, remove_filer=True)
print(f"WER of all tasks = {res}")

WER of Arg_Oly = 0.1881633965905436


WER of Cartoon = 0.1891544590079671
WER of RtSwithoutRAA = 0.24019138755980862
WER of RtSwithRAA = 0.2592875318066158
WER of monologue task = 0.22131039046988749
WER of WoZ_Interview = 0.2485251852972319
WER of all tasks = 0.22959348096312324


The following code block counts disfluency tags in beginners' speech.

In [22]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = beginner_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<DISFLUENCY>")
print(f"[Manual] N_disfluency of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_disfluency of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = beginner_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_disfluency of Arg_Oly = 549
[Automatic] N_disfluency of Arg_Oly = 498
[Manual] N_disfluency of Cartoon = 788
[Automatic] N_disfluency of Cartoon = 694
[Manual] N_disfluency of RtSwithoutRAA = 821
[Automatic] N_disfluency of RtSwithoutRAA = 732
[Manual] N_disfluency of RtSwithRAA = 698
[Automatic] N_disfluency of RtSwithRAA = 672
[Manual] N_disfluency of monologue task = 2856
[Automatic] N_disfluency of monologue task = 2596
[Manual] N_disfluency of WoZ_Interview = 955
[Automatic] N_disfluency of WoZ_Interview = 730


The following code block counts MCP tags in beginners' speech.

In [23]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = beginner_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CI>")
print(f"[Manual] N_MCP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_MCP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = beginner_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_MCP of Arg_Oly = 1251
[Automatic] N_MCP of Arg_Oly = 1291
[Manual] N_MCP of Cartoon = 1603
[Automatic] N_MCP of Cartoon = 1740
[Manual] N_MCP of RtSwithoutRAA = 1763
[Automatic] N_MCP of RtSwithoutRAA = 1972
[Manual] N_MCP of RtSwithRAA = 1687
[Automatic] N_MCP of RtSwithRAA = 1867
[Manual] N_MCP of monologue task = 6304
[Automatic] N_MCP of monologue task = 6870
[Manual] N_MCP of WoZ_Interview = 1913
[Automatic] N_MCP of WoZ_Interview = 2069


The following code block counts ECP tags in beginners' speech.

In [24]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = beginner_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CE>")
print(f"[Manual] N_ECP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_ECP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = beginner_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_ECP of Arg_Oly = 254
[Automatic] N_ECP of Arg_Oly = 213
[Manual] N_ECP of Cartoon = 438
[Automatic] N_ECP of Cartoon = 378
[Manual] N_ECP of RtSwithoutRAA = 421
[Automatic] N_ECP of RtSwithoutRAA = 383
[Manual] N_ECP of RtSwithRAA = 416
[Automatic] N_ECP of RtSwithRAA = 342
[Manual] N_ECP of monologue task = 1529
[Automatic] N_ECP of monologue task = 1316
[Manual] N_ECP of WoZ_Interview = 472
[Automatic] N_ECP of WoZ_Interview = 443


### 5.2. Intemediate

The following code block loads intermediate group's speech.

In [25]:
intemediate_dataset = load_dataset(rating_filter_monologue=[3, 4, 5], rating_filter_dialogue=[2, 3])

The following code block calculates WER of intemediate learners' speech.

In [26]:
monologue_data = []
for monologue_task in MONOLOGUE_TASK:
    annotation_results = intemediate_dataset["monologue"][monologue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {monologue_task} = {res}")

    monologue_data += annotation_results

res = calculate_wer(monologue_data, remove_filer=True)
print(f"WER of monologue task = {res}")

dialogue_data = []
for dialogue_task in DIALOGUE_TASK:
    annotation_results = intemediate_dataset["dialogue"][dialogue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {dialogue_task} = {res}")

    dialogue_data += annotation_results

all_task_data = monologue_data + dialogue_data

res = calculate_wer(all_task_data, remove_filer=True)
print(f"WER of all tasks = {res}")

WER of Arg_Oly = 0.16628927257288423
WER of Cartoon = 0.13612896428195131
WER of RtSwithoutRAA = 0.21240377317575626
WER of RtSwithRAA = 0.20619413843275827
WER of monologue task = 0.18106321676741427
WER of WoZ_Interview = 0.14034321645342998
WER of all tasks = 0.165068692863146


The following code block counts disfluency tags in intemediate learners' speech.

In [27]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = intemediate_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<DISFLUENCY>")
print(f"[Manual] N_disfluency of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_disfluency of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = intemediate_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_disfluency of Arg_Oly = 961
[Automatic] N_disfluency of Arg_Oly = 936
[Manual] N_disfluency of Cartoon = 1430
[Automatic] N_disfluency of Cartoon = 1333
[Manual] N_disfluency of RtSwithoutRAA = 1400
[Automatic] N_disfluency of RtSwithoutRAA = 1209
[Manual] N_disfluency of RtSwithRAA = 1418
[Automatic] N_disfluency of RtSwithRAA = 1258
[Manual] N_disfluency of monologue task = 5209
[Automatic] N_disfluency of monologue task = 4736
[Manual] N_disfluency of WoZ_Interview = 2662
[Automatic] N_disfluency of WoZ_Interview = 2439


The following code block counts MCP tags in intemediate learners' speech.

In [28]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = intemediate_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CI>")
print(f"[Manual] N_MCP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_MCP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = intemediate_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_MCP of Arg_Oly = 2181
[Automatic] N_MCP of Arg_Oly = 2536
[Manual] N_MCP of Cartoon = 2971
[Automatic] N_MCP of Cartoon = 3665
[Manual] N_MCP of RtSwithoutRAA = 3041
[Automatic] N_MCP of RtSwithoutRAA = 3720
[Manual] N_MCP of RtSwithRAA = 3251
[Automatic] N_MCP of RtSwithRAA = 3882
[Manual] N_MCP of monologue task = 11444
[Automatic] N_MCP of monologue task = 13803
[Manual] N_MCP of WoZ_Interview = 4911
[Automatic] N_MCP of WoZ_Interview = 7022


The following code block counts ECP tags in intermediate learners' speech.

In [29]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = intemediate_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CE>")
print(f"[Manual] N_ECP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_ECP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = intemediate_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_ECP of Arg_Oly = 589
[Automatic] N_ECP of Arg_Oly = 508
[Manual] N_ECP of Cartoon = 1041
[Automatic] N_ECP of Cartoon = 952
[Manual] N_ECP of RtSwithoutRAA = 919
[Automatic] N_ECP of RtSwithoutRAA = 779
[Manual] N_ECP of RtSwithRAA = 942
[Automatic] N_ECP of RtSwithRAA = 808
[Manual] N_ECP of monologue task = 3491
[Automatic] N_ECP of monologue task = 3047
[Manual] N_ECP of WoZ_Interview = 1682
[Automatic] N_ECP of WoZ_Interview = 1813


### 5.3. Advanced 

The following code block loads advanced learners' speech.

In [30]:
advanced_dataset = load_dataset(rating_filter_monologue=[6, 7, 8], rating_filter_dialogue=[4, 5])

The following code block calculates WER of advanced learners' speech.

In [31]:
monologue_data = []
for monologue_task in MONOLOGUE_TASK:
    annotation_results = advanced_dataset["monologue"][monologue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {monologue_task} = {res}")

    monologue_data += annotation_results

res = calculate_wer(monologue_data, remove_filer=True)
print(f"WER of monologue task = {res}")

dialogue_data = []
for dialogue_task in DIALOGUE_TASK:
    annotation_results = advanced_dataset["dialogue"][dialogue_task]

    res = calculate_wer(annotation_results, remove_filer=True)

    print(f"WER of {dialogue_task} = {res}")

    dialogue_data += annotation_results

all_task_data = monologue_data + dialogue_data

res = calculate_wer(all_task_data, remove_filer=True)
print(f"WER of all tasks = {res}")

WER of Arg_Oly = 0.13107014388489208
WER of Cartoon = 0.10677466863033873
WER of RtSwithoutRAA = 0.14236410698878343
WER of RtSwithRAA = 0.15546587120547464
WER of monologue task = 0.13446243099092356
WER of WoZ_Interview = 0.081675562024907
WER of all tasks = 0.12261857241354283


The following code block counts disfluency tags in advanced learners' speech.

In [32]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = advanced_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<DISFLUENCY>")
print(f"[Manual] N_disfluency of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_disfluency of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = advanced_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<DISFLUENCY>")

    print(f"[Manual] N_disfluency of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_disfluency of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_disfluency of Arg_Oly = 366
[Automatic] N_disfluency of Arg_Oly = 359
[Manual] N_disfluency of Cartoon = 595
[Automatic] N_disfluency of Cartoon = 490
[Manual] N_disfluency of RtSwithoutRAA = 713
[Automatic] N_disfluency of RtSwithoutRAA = 591
[Manual] N_disfluency of RtSwithRAA = 771
[Automatic] N_disfluency of RtSwithRAA = 626
[Manual] N_disfluency of monologue task = 2445
[Automatic] N_disfluency of monologue task = 2066
[Manual] N_disfluency of WoZ_Interview = 318
[Automatic] N_disfluency of WoZ_Interview = 345


The following code block counts MCP tags in advanced learners' speech.

In [33]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = advanced_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CI>")
print(f"[Manual] N_MCP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_MCP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = advanced_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CI>")

    print(f"[Manual] N_MCP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_MCP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_MCP of Arg_Oly = 870
[Automatic] N_MCP of Arg_Oly = 1096
[Manual] N_MCP of Cartoon = 985
[Automatic] N_MCP of Cartoon = 1334
[Manual] N_MCP of RtSwithoutRAA = 1342
[Automatic] N_MCP of RtSwithoutRAA = 1657
[Manual] N_MCP of RtSwithRAA = 1394
[Automatic] N_MCP of RtSwithRAA = 1753
[Manual] N_MCP of monologue task = 4591
[Automatic] N_MCP of monologue task = 5840
[Manual] N_MCP of WoZ_Interview = 750
[Automatic] N_MCP of WoZ_Interview = 1231


The following code block counts ECP tags in advanced learners' speech.

In [34]:
for monologue_task in MONOLOGUE_TASK:
    annotation_results = advanced_dataset["monologue"][monologue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {monologue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {monologue_task} = {sum(n_tags_auto)}")

n_tags_manu, n_tags_auto = count_tags(monologue_data, "<CE>")
print(f"[Manual] N_ECP of monologue task = {sum(n_tags_manu)}")
print(f"[Automatic] N_ECP of monologue task = {sum(n_tags_auto)}")

for dialogue_task in DIALOGUE_TASK:
    annotation_results = advanced_dataset["dialogue"][dialogue_task]

    n_tags_manu, n_tags_auto = count_tags(annotation_results, "<CE>")

    print(f"[Manual] N_ECP of {dialogue_task} = {sum(n_tags_manu)}")
    print(f"[Automatic] N_ECP of {dialogue_task} = {sum(n_tags_auto)}")

[Manual] N_ECP of Arg_Oly = 299
[Automatic] N_ECP of Arg_Oly = 280
[Manual] N_ECP of Cartoon = 506
[Automatic] N_ECP of Cartoon = 480
[Manual] N_ECP of RtSwithoutRAA = 492
[Automatic] N_ECP of RtSwithoutRAA = 461
[Manual] N_ECP of RtSwithRAA = 484
[Automatic] N_ECP of RtSwithRAA = 423
[Manual] N_ECP of monologue task = 1781
[Automatic] N_ECP of monologue task = 1644
[Manual] N_ECP of WoZ_Interview = 260
[Automatic] N_ECP of WoZ_Interview = 390
